<a href="https://colab.research.google.com/github/hyuna0926/RecommendSystem/blob/main/test/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 필요 라이브러리 및 데이터 로드

In [ ]:
import pandas as pd
import numpy as np
import warnings
# 경고 제거
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from scipy import sparse
from scipy.sparse import csr_matrix
from tqdm.notebook import tqdm
import numpy as np

# from implicit.als import AlternatingLeastSquares as ALS
# import implicit
import random
import os

In [ ]:
product=pd.read_parquet('/content/drive/MyDrive/CP2_Phase2/product.parquet')

In [ ]:
df= pd.read_parquet('/content/drive/MyDrive/CP2_Phase2/cart_purchase.parquet')
df.head()

,created_at,customer_id,product_id,p_idx,event,c_idx
0,2018-07-29 15:22:01,5868,54728,29317,purchase,3039
1,2021-01-15 19:52:40,82831,54728,29317,purchase,41968
2,2016-10-20 10:16:35,82831,34982,8394,purchase,41968
3,2018-08-26 14:09:52,82831,8627,9052,purchase,41968
4,2018-03-17 15:08:08,82831,46703,23994,purchase,41968


In [ ]:
df['values']=1

- 가장 많이 팔린 상품 순으로 정렬

In [ ]:
p = df.groupby('p_idx').sum()['values'].to_frame().reset_index()

- 데이터 분리

In [ ]:
test = df.groupby('c_idx').sample(frac=0.2,random_state=42)
train = df.drop(test.index)

In [ ]:
df_group=df.groupby(['c_idx'], as_index = False).count()

upper = df[df['c_idx'].isin(df_group.query("values >= 20").c_idx)]


test = upper.groupby('c_idx').sample(frac=0.2,random_state=42)
train = upper.drop(test.index)

user_20 = sparse.csr_matrix((train['values'],(train['c_idx'], train['p_idx'])))
test_user_20 = sparse.csr_matrix((test['values'],(test['c_idx'], test['p_idx'])))

In [ ]:
# 검증셋 만들기
test_df = test.groupby('c_idx')['p_idx'].unique().to_frame().reset_index()

# 성능 평가

## precision@k

In [ ]:
def precision_at_k(user_id,data,k=25):
  
  count=0
  user_buy = data[data['c_idx']==user_id].p_idx
  top_k = p.sort_values('values').tail(25).p_idx
  for i in user_buy:
    for j in top_k:
      if i==j:
        count+=1
  
  precision = count/len(top_k)
  return precision

In [ ]:
def mean_precision_k(k,data):
  precision=0
  total =0
  for i in range(1,k+1):
    #train,test셋으로 분리했기 때문에 없는 user_id는 넘김
    if i in upper_20:
      total+=1
      precision+=precision_at_k(i,data,25)
    else:
      pass
  mean_precision = precision/total
  return mean_precision

In [ ]:
mean_precision_k(49000,train)

0.05392203953445

In [ ]:
mean_precision_k(50000,test)

0.01389942515729004

## hit rate@k

In [ ]:
def hit_rate(k):
  total= 0
  user = 0
  for c_idx in range(k):
    count=0 # for문 돌 때마다 리셋
    if c_idx in upper_20:
      top_k = p.sort_values('values').tail(25).p_idx
      buy_test = test_df[test_df['c_idx']==c_idx].p_idx.values[0]
      user+=1
      for i in buy_test:
        for j in top_k:
          if i==j:
            count+=1
    if count >=1: #count가 1 이상이면 hit한 것이니까 전체에 1 추가
      total+=1
    
  hit_rate = total/user

  return hit_rate

In [ ]:
hit_rate(50000)

0.2644609396216167